<a href="https://colab.research.google.com/github/tasnimislam/TeamMeena/blob/main/Data_Imbalancing_handeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Libraries

In [ ]:
!pip install wfdb==0.1.2

import pandas as pd
import os
import wfdb
from wfdb import rdsamp
from tqdm import tqdm
import random
import numpy as np
from scipy.ndimage import filters

     |████████████████████████████████| 163kB 20.2MB/s 
  Created wheel for wfdb: filename=wfdb-0.1.2-cp36-none-any.whl size=23148 sha256=38b61512daf893254ca1bffd6fa17f0cf6e4a4c9d09fa82a24fe7957dbdb1208
  Stored in directory: /root/.cache/pip/wheels/b9/2e/12/e215b6517234c8fc17638fba70081e319ec8f0c21e67c1619b
Successfully built wfdb


# Input data Preparation


In [ ]:
keras_input = pd.DataFrame()
file = open("/content/drive/MyDrive/mmg_dataset_v2/fvl files/0_3 theke 3 hertz.txt")
i = 0
# Record   | Chann | Gestation | Rec. time |   Group   |   RMS   |  Fmed  | Fpeak  | Samp. en. | Premature | Early 
for line in file:
  if i<2:
    i+=1
    continue
  line = line.split("|")

  keras_input.at[i, "Record"] = line[0]
  keras_input.at[i, "Channel_no"] = line[1]
  keras_input.at[i, "Gestation"] = line[2]
  keras_input.at[i, "Record_time"] = line[3]
  keras_input.at[i, "RMS"] = line[5]
  keras_input.at[i, "Fmed"] = line[6]
  keras_input.at[i, "Fpeak"] = line[7]
  keras_input.at[i, "Samp. en."] = line[8]
  keras_input.at[i, "Premature"] = line[9]
  keras_input.at[i, "Early"] = line[10][:-1]
  i +=1
keras_input.head()

,Record,Channel_no,Gestation,Record_time,RMS,Fmed,Fpeak,Samp. en.,Premature,Early
2,tpehg1007,1,35.00,31.29,16.5069,0.3688,0.3324,0.5180,t,f
3,tpehg1007,2,35.00,31.29,16.9946,0.3688,0.3159,0.3810,t,f
4,tpehg1007,3,35.00,31.29,8.4198,0.3981,0.2938,0.5640,t,f
5,tpehg1021,1,38.57,22.29,14.4062,0.4170,0.3828,0.4460,f,t
6,tpehg1021,2,38.57,22.29,12.3715,0.4120,0.3136,0.4370,f,t


In [ ]:
keras_input["Channel_no"] = keras_input.Channel_no.astype("int64")
print(len(keras_input))
listi = list(keras_input.columns)

900


In [ ]:
signals = []
classi_list = []
read_file_dir = "/content/drive/MyDrive/mmg_dataset_v2/term-preterm-ehg-database-1.0.1.zip (Unzipped Files)/term-preterm-ehg-database-1.0.1/tpehgdb"
for i in tqdm(range(len(keras_input))):
  record_name = keras_input.iloc[i]["Record"][1:-1]
  record_name = os.path.join(read_file_dir, record_name + ".hea")
  try:
    sig, fields = rdsamp(record_name[:-4], sampfrom=0, sampto=[], channels=[], physical=1, stacksegments=1, pbdl=0, keepfiles=0)
    if keras_input.iloc[i]["Channel_no"] == 1.0: signal = sig[:,2] 
    if keras_input.iloc[i]["Channel_no"] == 2.0: signal = sig[:,6]
    if keras_input.iloc[i]["Channel_no"] == 3.0: signal = sig[:,10]
    if keras_input.iloc[i]["Premature"]== keras_input.iloc[0]["Premature"]: classi = 0
    if keras_input.iloc[i]["Premature"]==keras_input.iloc[3]["Premature"]: classi = 1
    signals.append(signal)
    classi_list.append(classi)
  except:
    continue

100%|██████████| 900/900 [00:07<00:00, 112.72it/s]


In [ ]:
print(len(classi_list)) 

603


In [ ]:
#there were faulty file names so we are working on remaining 603 signals
term = classi_list.count(1)
preterm = classi_list.count(0)
print("preterm", preterm)
print("term", term)

preterm 75
term 528


Preterm and Term is imbalanced, so we balance the dataset, oversampling the dataset using linear oversampling method


In [ ]:
def linear_augment_function(signal, no_sample, cut_shape):
    '''
    signal: numpy formatted 1D single channel signal
    no_sample: the number of signals desired to be augmented for a single signal
    cut_shape: the range of signal that will be cut for further concatenation
    returns a list of augmented signals of a particular signal of particular channel
    '''
    list_aug_signals = []
    sig_shape = signal.shape[0]
    last_range_lim = sig_shape-2*cut_shape
    for i in range(no_sample):
        x = random.randrange(0, last_range_lim, 1)
        first_part_numpy_signal = signal[x:x+cut_shape]
        numpy_sig = np.concatenate((signal, first_part_numpy_signal))
        new_signal = numpy_sig[:sig_shape]
        filterd_signal = filters.convolve1d(np.array(signal), np.array(new_signal))
        filterd_signal = filterd_signal[:sig_shape]
        list_aug_signals.append(filterd_signal)
    return list_aug_signals

In [ ]:
preterm_index = [i for i, value in enumerate(classi_list) if value == 0]
term_index = [i for i, value in enumerate(classi_list) if value == 1]
print(preterm_index)

[0, 1, 2, 102, 103, 104, 129, 130, 131, 132, 133, 134, 207, 208, 209, 219, 220, 221, 225, 226, 227, 279, 280, 281, 297, 298, 299, 303, 304, 305, 327, 328, 329, 333, 334, 335, 336, 337, 338, 348, 349, 350, 360, 361, 362, 384, 385, 386, 387, 388, 389, 402, 403, 404, 447, 448, 449, 555, 556, 557, 558, 559, 560, 570, 571, 572, 591, 592, 593, 597, 598, 599, 600, 601, 602]


In [ ]:
augmented_final_preterm = []
for i in tqdm(preterm_index):
  augmented_final_preterm.append(linear_augment_function(signals[i], 10, 5000))

100%|██████████| 75/75 [22:03<00:00, 17.65s/it]


In [ ]:
print(len(augmented_final_preterm[0]))
augmented_final_preterm[0]

10


[array([ 0.14316457,  0.24252818,  0.33846775, ..., -0.14777497,
        -0.21064255, -0.27229768]),
 array([ 0.14316457,  0.24252818,  0.33846775, ..., -0.14777497,
        -0.21064255, -0.27229768]),
 array([ 0.14316457,  0.24252818,  0.33846775, ..., -0.14777497,
        -0.21064255, -0.27229768]),
 array([ 0.14316457,  0.24252818,  0.33846775, ..., -0.14777497,
        -0.21064255, -0.27229768]),
 array([ 0.14316457,  0.24252818,  0.33846775, ..., -0.14777497,
        -0.21064255, -0.27229768]),
 array([ 0.14316457,  0.24252818,  0.33846775, ..., -0.14777497,
        -0.21064255, -0.27229768]),
 array([ 0.14316457,  0.24252818,  0.33846775, ..., -0.14777497,
        -0.21064255, -0.27229768]),
 array([ 0.14316457,  0.24252818,  0.33846775, ..., -0.14777497,
        -0.21064255, -0.27229768]),
 array([ 0.14316457,  0.24252818,  0.33846775, ..., -0.14777497,
        -0.21064255, -0.27229768]),
 array([ 0.14316457,  0.24252818,  0.33846775, ..., -0.14777497,
        -0.21064255, -0.27

In [ ]:
listi = []
for aug in augmented_final_preterm:
  for all in aug:
    listi.append(all)

In [ ]:
print(len(listi))
preterm_after_augment = listi

750


In [ ]:
augmented_final_term = []
for i in tqdm(term_index):
  augmented_final_term.append(linear_augment_function(signals[i], 3, 5000))

100%|██████████| 528/528 [29:22<00:00,  3.34s/it]


In [ ]:
listi_term = []
for aug in augmented_final_term:
  for all in aug:
    listi_term.append(all)

In [ ]:
print(len(listi_term))

1584


In [ ]:
print(1584+750)

2334


## Fix the shape of the signal

In [ ]:
signals_all = preterm_after_augment + listi_term
list_signal_len = []
for signal in signals_all:
  list_signal_len.append(len(signal))
print(min(list_signal_len))

24105


The signal is ready for training